# Compile TrackMate Data

Run this script to compile each data metric from TrackMate results files. Choose directory that contains all your trackmate results files (export.csv) in individual folders. 

In [ ]:
import os
import pandas as pd
from tkinter import filedialog


# Ask the user for track duration to filter out
track_duration_input = int(input("Enter the minimum track duration (sec) you would like (Please enter whole number): "))

main_directory = filedialog.askdirectory(title="Select Directory Containing Condition Folders")

print(main_directory)

# Check if a directory was selected
if not main_directory:
    print("No directory selected. Exiting...")
    exit()

# All metrics from trackmate export.csv files
column_titles = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'NUMBER_SPLITS', 'NUMBER_MERGES',
                 'NUMBER_COMPLEX', 'LONGEST_GAP', 'TRACK_START', 'TRACK_STOP', 
                 'TRACK_DISPLACEMENT', 'TRACK_DURATION', 'TRACK_MEAN_SPEED', 
                 'TRACK_MAX_SPEED', 'TRACK_MIN_SPEED', 'TRACK_MEDIAN_SPEED', 
                 'TRACK_STD_SPEED', 'TRACK_MEAN_QUALITY', 'TOTAL_DISTANCE_TRAVELED', 
                 'MAX_DISTANCE_TRAVELED', 'CONFINEMENT_RATIO', 'MEAN_STRAIGHT_LINE_SPEED',
                 'LINEARITY_OF_FORWARD_PROGRESSION', 'MEAN_DIRECTIONAL_CHANGE_RATE',
                 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION']

# Create the trackmate_results folder in the chosen directory
trackmate_results_folder = os.path.join(main_directory, 'trackmate_results')
if not os.path.exists(trackmate_results_folder):
    os.makedirs(trackmate_results_folder)

# Create a dictionary to hold DataFrames for each column
df_dict = {title: pd.DataFrame() for title in column_titles}

# Iterate over folders in the main directory
for folder_idx, folder_name in enumerate(os.listdir(main_directory)):
    folder_path = os.path.join(main_directory, folder_name)
    
    if not os.path.isdir(folder_path):
        continue
    
    # Check if export.csv exists in the folder
    export_csv_path = os.path.join(folder_path, 'export.csv')
    if not os.path.exists(export_csv_path):
        print(f"export.csv not found in folder '{folder_name}'. Skipping...")
        continue
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(export_csv_path)

    # Convert columns to numeric values
    df[column_titles] = df[column_titles].apply(pd.to_numeric, errors='coerce')

    # Filter rows based on conditions
    filtered_rows = df.loc[(~df[column_titles].isna().any(axis=1)) & (df['TRACK_DURATION'] >= track_duration_input)]

    # Add filtered rows to the dictionary
    for column in column_titles:
        if not filtered_rows[column].isna().all():
            df_dict[column] = pd.concat([df_dict[column], filtered_rows[[column]].reset_index(drop=True)], axis=1)

# Save the raw DataFrames to separate Excel files for each condition with each metric in a different sheet
with pd.ExcelWriter(os.path.join(trackmate_results_folder, "compiled_raw.xlsx")) as writer:
    for title, df in df_dict.items():
        # Truncate the sheet name if it exceeds 31 characters
        sheet_name = title[:31] if len(title) > 31 else title
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Data saved to {os.path.join(trackmate_results_folder, 'compiled_raw.xlsx')}")
